# 1. Getting rcency frequency and monetary data 

In [1]:
import pyodbc
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

conn = pyodbc.connect('DRIVER=SQL Server;\
                       Server=localhost\sqlexpress;\
                       Database=Northwind;\
                       Trusted_Connection=Yes;')
cursor = conn.cursor()


In [2]:
#importing customer segment data
df_customer = pd.read_sql_query('Select * from Northwind..rfm_customer_analysis',conn)
df_customer.head()

,custid,recency,frequency,monetary,r_score,f_score,m_score,rfm_score,segment
0,CENTC,661,1,100.8,1,1,1,111,Lost Customers
1,LAZYK,353,2,357.0,1,1,1,111,Lost Customers
2,LAUGB,129,3,522.5,1,1,1,111,Lost Customers
3,NORTS,11,3,649.0,3,1,1,311,New Customers
4,GALED,66,5,836.7,1,1,1,111,Lost Customers


In [3]:
#importing country segmentation data
df_country = pd.read_sql_query('Select * from Northwind..rfm_country_analysis',conn)
df_country.head()

,country,recency,frequency,monetary,r_score,f_score,m_score,rfm_score,segment
0,Poland,17,7,3531.95,1,1,1,111,Lost Customers
1,Norway,30,6,5735.15,1,1,1,111,Lost Customers
2,Argentina,12,16,8119.10,2,1,1,211,Promising
3,Portugal,32,13,12468.65,1,1,1,111,Lost Customers
4,Italy,10,28,16705.15,2,2,1,221,Promising


In [4]:
#getting geo data to visualize country in scatter map
df_geo = pd.read_html('https://developers.google.com/public-data/docs/canonical/countries_csv')[0]
df_geo.head()

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla


# 2. Data wranling

In [5]:
#replacing the inappropirate coding for US and UK
df_country['country'] = df_country['country'].replace({'UK':'United Kingdom','USA':'United States'}, regex=True)
df_country.head()

,country,recency,frequency,monetary,r_score,f_score,m_score,rfm_score,segment
0,Poland,17,7,3531.95,1,1,1,111,Lost Customers
1,Norway,30,6,5735.15,1,1,1,111,Lost Customers
2,Argentina,12,16,8119.10,2,1,1,211,Promising
3,Portugal,32,13,12468.65,1,1,1,111,Lost Customers
4,Italy,10,28,16705.15,2,2,1,221,Promising


In [6]:
#combining with the geo data
df_country = pd.merge(df_country, df_geo, how='left',left_on='country',right_on='name').drop(columns=['country_y','name'])
df_country.head()

,country_x,recency,frequency,monetary,r_score,f_score,m_score,rfm_score,segment,latitude,longitude
0,Poland,17,7,3531.95,1,1,1,111,Lost Customers,51.919438,19.145136
1,Norway,30,6,5735.15,1,1,1,111,Lost Customers,60.472024,8.468946
2,Argentina,12,16,8119.10,2,1,1,211,Promising,-38.416097,-63.616672
3,Portugal,32,13,12468.65,1,1,1,111,Lost Customers,39.399872,-8.224454
4,Italy,10,28,16705.15,2,2,1,221,Promising,41.871940,12.567380


# 3. Exploratory data analysis and visualization

Scatter plots examine the relationships between the three factors recency, frequency and monetary

In [12]:
#Recency vs. Monetary
fig = px.scatter(df_customer, 
                 x='monetary', 
                 y='recency',
                 hover_data=['custid','monetary','recency','segment'], 
                 color='segment',
                 labels={'recency':'Recency (days)','monetary':'Total sales ($)','segment':'Customer Segments'})

fig.update_layout(title_text='Recency (days) vs. Total sales ($)',title_x=0.5,
                  legend=dict(yanchor="top",y=0.98,xanchor="left",x=0.82))

fig.show()

In [15]:
#Frequency vs. Monetary
fig = px.scatter(df_customer, 
                 x='monetary', 
                 y='frequency',
                 hover_data=['custid','monetary','frequency','segment'],
                 color='segment',
                 labels={'monetary':'Total sales ($)','frequency':'Frequency','segment':'Customer Segments'})
fig.update_layout(title_text='Requency vs. Total sales ($)',title_x=0.5,
                  legend=dict(yanchor="top",y=0.98,xanchor="left",x=0.82))
fig.show()

In [16]:
#Recency vs. Frequency
fig = px.scatter(df_customer, 
                 x='frequency', 
                 y='recency',
                 hover_data=['custid','frequency','recency','segment'],
                 color='segment',
                 labels={'recency':'Recency','frequency':'Frequency','segment':'Customer Segments'})
fig.update_layout(title_text='Recency (days) vs. Frequency)',title_x=0.5,
                  legend=dict(yanchor="top",y=0.98,xanchor="left",x=0.82))
fig.show()

In [38]:
#calculating median for recency, frequency, and monetary for each segment
df_customer.groupby(['segment']).agg({'recency':'median','frequency':'median','monetary':'median'})

,recency,frequency,monetary
segment,,,
At Risk,63.5,7.0,6816.125
Champion,10.0,14.0,22607.700
Lost Customers,114.5,3.5,1529.950
Loyal Customer,25.5,13.5,21296.975
New Customers,11.5,4.5,1231.900
Others,11.0,7.0,4242.200
Potential Loyalist,57.5,9.5,16748.600
Promising,30.0,7.0,4596.200


In [17]:
#horizontal chart of the total salses by customer segments
fig = px.histogram(df_customer, 
                   x="monetary", 
                   y="segment",
                   orientation='h',
                   labels={'monetary':'Total sales','segment':'Customer Segments'},
                   text_auto=True
            )

fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [18]:
#treemap of segments by number of customrs
fig = px.treemap(df_customer,path=['segment'],width=900,height=600)
fig.data[0].textinfo = 'label+text+value+percent entry'
fig.show()

In [19]:
#treemap of segments by number of customrs
fig = px.treemap(df_customer,path=['segment'],values='monetary',width=900,height=600)
fig.data[0].textinfo = 'label+text+value+percent entry'
fig.show()

In [20]:
fig=px.scatter_mapbox(df_country, 
                      lat        = df_country['latitude'], 
                      lon        = df_country['longitude'],
                      color      = df_country['segment'],
                      size       = df_country['rfm_score'],
                      hover_name = 'country_x', 
                      hover_data = ['rfm_score'],
                      zoom       = 1)

fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(margin={'r':0,'t':50,'l':0,'b':10})
fig.show()